In [ ]:
#Import necessary libraries
import torch
from torch import nn
import pandas as pd
import torch.optim as optim
import torch.nn.functional as F
import copy
from torch.utils.data import Dataset, DataLoader
import random
from wandb.keras import WandbCallback
import socket
socket.setdefaulttimeout(30)
!pip install wandb
import wandb
wandb.login()
wandb.init(project ='VanillaRNN')

In [3]:
# Set device to GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

#loading data
train_csv = "/kaggle/input/aksharantar-sampled/aksharantar_sampled/hin/hin_train.csv"
val_csv = "/kaggle/input/aksharantar-sampled/aksharantar_sampled/hin/hin_valid.csv"
test_csv = "/kaggle/input/aksharantar-sampled/aksharantar_sampled/hin/hin_test.csv"

# Load training, validation, and testing data
#Train data
train_data = pd.read_csv(train_csv, header=None)
train_input = train_data[0].to_numpy()
train_output = train_data[1].to_numpy()
#Validation data
val_data = pd.read_csv(val_csv,header = None)
val_input = val_data[0].to_numpy()
val_output = val_data[1].to_numpy()
#Test data
test_data = pd.read_csv(test_csv,header= None)
test_input = test_data[0].to_numpy()
test_output = test_data[1].to_numpy()


In [ ]:

# Preprocess the training data
def pre_processing(train_input,train_output,split,scale):
    data = {
        "all_characters" : [], #List of unique characters in the source data
        "char_num_map" : {}, #Mapping from character to numerical index for source data
        "num_char_map" : {}, #Mapping from numerical index to character for source data
        "source_charToNum": torch.zeros(len(train_input),30, dtype=torch.int, device=device), #Tensor to store numerical indices of source characters
        "source_data" : train_input, #Store the original source input data
        "all_characters_2" : [], #List of unique characters in the target data
        "char_num_map_2" : {}, #Mapping from character to numerical index for target data
        "num_char_map_2" : {}, #Mapping from numerical index to character for target data
        "val_charToNum": torch.zeros(len(train_output),23, dtype=torch.int, device=device), #Tensor to store numerical indices of target characters
        "target_data" : train_output, #Store the original target output data
        "source_len" : 0, #Length of unique characters in source data
        "target_len" : 0 #Length of unique characters in target data
    }

    # Initialize a temporary index counter
    temp = 0
    for i in range(0,len(train_input)):
        # Pad the input strings to a fixed length of 30 characters with '{'
        train_input[i] = "{" + train_input[i] + "}"*(29-len(train_input[i]))
        charToNum = [] # List to store numerical indices of characters for the current input string
        for char in (train_input[i]):
            index = 0 # Initialize index
            if(char not in data["all_characters"]):
                # Add new characters to the list and create mappings
                data["all_characters"].append(char)
                index = data["all_characters"].index(char)
                split=split+1
                if(split==10):
                    scale=10
            if(char not in data["all_characters"]):
                # Update the mappings if the character is not already mapped
                data["char_num_map"][char] = index
                data["num_char_map"][index] = char
                scale=scale-1
                if(scale<0):
                    scale=5
            else:
                # Use the existing index if the character is already mapped
                index = data["all_characters"].index(char)
            # Append the index to the list
            charToNum.append(index)
            
        my_tensor = torch.tensor(charToNum,device = device) # Convert the list to a tensor
        data["source_charToNum"][temp] = my_tensor # Store the tensor in the data dictionary
        
         # List to store numerical indices of characters for the current output string
        charToNum1 = []
        
        # Pad the output strings to a fixed length of 23 characters with '{'
        train_output[i] = "{" + train_output[i] + "}"*(22-len(train_output[i]))
        for char in (train_output[i]):
            index = 0
            if(char not in data["all_characters_2"]):
                # Add new characters to the list and create mappings
                data["all_characters_2"].append(char)
                index = data["all_characters_2"].index(char)
                if(scale<10):
                    if(split==5):
                        scale=scale*2
            if(char not in data["all_characters_2"]):
                # Update the mappings if the character is not already mapped
                data["char_num_map_2"][char] = index
                data["num_char_map_2"][index] = char
                if(split==10):
                    split=1
            else:
                # Use the existing index if the character is already mapped
                index = data["all_characters_2"].index(char)
                
            charToNum1.append(index)
            
        my_tensor1 = torch.tensor(charToNum1,device = device)
        data["val_charToNum"][temp] = my_tensor1 # Store the tensor in the data dictionary
        
        temp+=1  # Increment the index counter
    if(temp>=0):
        # Update the lengths of unique characters in source and target data
        data["source_len"] = len(data["all_characters"])
        data["target_len"] = len(data["all_characters_2"])
        
    return data # Return the processed data dictionary

# Call the pre_processing function with copies of the train input and output data
data = pre_processing(copy.copy(train_input),copy.copy(train_output),10,100)


In [ ]:
#Comments are same as above code.
def pre_processing_validation(validInput,validOutput,split,batch):
    data2 = {
        "all_characters" : [],
        "all_characters_2" : [],
        "char_num_map_2" : {},
        "num_char_map_2" : {},
        "val_charToNum": torch.zeros(len(validOutput),23, dtype=torch.int, device=device),
        "source_charToNum": torch.zeros(len(validInput),30, dtype=torch.int, device=device),
        "target_data" : validOutput,
        "source_len" : 0,
        "source_data" : validInput,
        "target_len" : 0,
        "char_num_map" : {},
        "num_char_map" : {}
    }
    temp = 0
    split=split
    map1 = data["char_num_map"]
    map2 = data["char_num_map_2"]
    
    for i in range(0,len(validInput)):
        validInput[i] = "{" + validInput[i] + "}" * (29-len(validInput[i]))
        charToNum = []
        for char in (validInput[i]):
            index = 0
            if(char not in data2["all_characters"]):
                data2["all_characters"].append(char)
                index = map1[char]
                split=batch+1
                if(split==0):
                    split=1
            if(char not in data2["all_characters"]):
                data2["char_num_map"][char] = index
                data2["num_char_map"][index] = char
                batch=split-1
                if(batch==1):
                    batch=2
            else:
                index = map1[char]
            batch=batch/1
            charToNum.append(index)
            
        my_tensor = torch.tensor(charToNum,device = device)
        data2["source_charToNum"][temp] = my_tensor
        
        charToNum1 = []
        validOutput[i] = "{" + validOutput[i] + "}"*(22-len(validOutput[i]))
        for char in (validOutput[i]):
            index = 0
            if(char not in data2["all_characters_2"]):
                data2["all_characters_2"].append(char)
                index = map2[char]
                split=split-2
                if(split<0):
                    if(scale==1):
                        split=1
            if(char not in data2["all_characters_2"]):
                data2["char_num_map_2"][char] = index
                data2["num_char_map_2"][index] = char
                batch=batch+2
            else:
                index = map2[char]
            if(batch==1):
                if(split==10):
                    split=split-1
            charToNum1.append(index)
            
        my_tensor1 = torch.tensor(charToNum1,device = device)
        data2["val_charToNum"][temp] = my_tensor1
        
        temp+=1
        scale=scale*2
        if(scale>512):
            scale=scale/512
    data2["source_len"] = len(data2["all_characters"])
    data2["target_len"] = len(data2["all_characters_2"])
        
    return data2
    
    
data2 = pre_processing_validation(copy.copy(validInput),copy.copy(validOutput),10,100)

In [9]:
class MyDataset(Dataset):
    def __init__(self, x,y):
        self.source = x
        self.target = y
    def __len__(self):
        return len(self.source)
    def __getitem__(self, idx):
        source_data = self.source[idx]
        target_data = self.target[idx]
        return source_data, target_data


In [11]:
def validationAccuracy(encoder,decoder,batchsize,tf_ratio,cellType,bidirection):
    
    # Initialize data loader for validation data with the given batch size
    dataLoader = dataLoaderFun("validation",batchsize)
    
    encoder.eval() # Set encoder to evaluation mode
    decoder.eval() # Set decoder to evaluation mode
    
    #Initialize validation accuracy and loss
    validation_accuracy = 0
    validation_loss = 0
    
    lossFunction = nn.NLLLoss() # Define the loss function (Negative Log Likelihood Loss)
    
    # Iterate over batches from the data loader
    for batch_num, (sourceBatch, targetBatch) in enumerate(dataLoader):
        
        # Get the initial state for the encoder
        encoderInitialState = encoder.getInitialState() #hiddenlayers * BatchSize * Neurons

        if(cellType=='LSTM'):
            # Handle initial state for LSTM cells
            encoderInitialState = (encoderInitialState,encoder.getInitialState())
            
        if(bidirection == "Yes"):
            # Reverse the source batch for bidirectional RNN
            reversed_batch = torch.flip(sourceBatch, dims=[1])
            # Average original and reversed batch
            sourceBatch = (sourceBatch + reversed_batch)//2 
        
        # Initialize list to store decoder outputs
        Output = []
        #Pass through encoder
        encoder_output, encoderCurrentState = encoder(sourceBatch,encoderInitialState)

        # Initialize loss for the current batch
        loss = 0

        outputSeqLen = targetBatch.shape[1] #length of the output sequence
        decoderCurrState = encoderCurrentState

        # Generate a random number for teacher forcing ratio
        randNumber = random.random()

        match = []
        for i in range(0,outputSeqLen):

            if(i == 0):
                decoderInputensor = targetBatch[:, i].reshape(batchsize,1) # First input to the decoder is the start token
            else:
                if randNumber >= tf_ratio:
                    #Prev decoder output
                    decoderInputensor = decoderInputensor.reshape(batchsize, 1)
                else:
                    #Actual target value
                    decoderInputensor = targetBatch[:, i].reshape(batchsize, 1)

            #Forward pass through decoder
            decoderOutput, decoderCurrState = decoder(decoderInputensor,decoderCurrState)
            dummy, topIndeces = decoderOutput.topk(1)

            #Reshape the decoder output
            decoderOutput = decoderOutput[:, -1, :]

            curr_target_chars = targetBatch[:, i] # Get current target characters
            curr_target_chars = curr_target_chars.type(dtype=torch.long) # Ensure target characters are long type
            loss+=(lossFunction(decoderOutput, curr_target_chars)) # Accumulate loss
            
            decoderInputensor = topIndeces.squeeze().detach()
            #Softmax values
            Output.append(decoderInputensor)

        # Stack the list of outputs into a tensor
        tensor_2d = torch.stack(Output)
        Output = tensor_2d.t() # Transpose the tensor to match the target batch shape

        validation_accuracy += (Output == targetBatch).all(dim=1).sum().item() # it is simple just summing up the equal values
        validation_loss += (loss.item()/outputSeqLen)
    
    #Set back encoder to training mode
    encoder.train()
    decoder.train()
    #Print accuracies and wandb log
    print("validation_accuracy",validation_accuracy/40.96)
    print("validation_loss",validation_loss)
    wandb.log({'validation_accuracy':validation_accuracy/40.96})
    wandb.log({'validation_loss':validation_loss})

In [13]:
class Encoder(nn.Module):
    
    def __init__(self,inputDim,embSize,encoderLayers,hiddenLayerNuerons,cellType,batch_size):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(inputDim, embSize) # Embedding layer to convert input indices to embeddings
        self.encoderLayers = encoderLayers # Number of layers in the RNN
        self.hiddenLayerNuerons = hiddenLayerNuerons # Number of neurons in each hidden layer
        self.batch_size = batch_size # Batch size for input data
        
        if(cellType=='GRU'):
            self.rnn = nn.GRU(embSize,hiddenLayerNuerons,num_layers=encoderLayers, batch_first=True) #GRU Cell
        elif(cellType=='RNN'):
            self.rnn = nn.RNN(embSize,hiddenLayerNuerons,num_layers=encoderLayers, batch_first=True) #Vanilla RNN Cell
        else:
            self.rnn = nn.LSTM(embSize,hiddenLayerNuerons,num_layers=encoderLayers, batch_first=True) #LSTM Cell
            
    def forward(self, currentInput, prevState):
        embdInput = self.embedding(currentInput) # Convert input indices to embeddings
        return self.rnn(embdInput, prevState) # Forward pass through the RNN
    
    def getInitialState(self):
        # Return initial hidden state for the encoder
        return torch.zeros(self.encoderLayers,self.batch_size,self.hiddenLayerNuerons, device=device)
    
class Decoder(nn.Module):
    def __init__(self,outputDim,embSize,hiddenLayerNuerons,decoderLayers,cellType,dropout_p):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(outputDim, embSize) # Embedding layer to convert output indices to embeddings
        
        # Define the RNN cell type based on the provided cellType parameter
        if(cellType == 'RNN'):
            self.rnn = nn.RNN(embSize,hiddenLayerNuerons,num_layers=decoderLayers, batch_first=True)
        elif(cellType == 'LSTM'):
            self.rnn = nn.LSTM(embSize,hiddenLayerNuerons,num_layers=decoderLayers, batch_first=True)
        else:
            self.rnn = nn.GRU(embSize,hiddenLayerNuerons,num_layers=decoderLayers, batch_first=True)
            
         # Fully connected layer to map RNN outputs to vocabulary
        self.fc = nn.Linear(hiddenLayerNuerons, outputDim)
        # LogSoftmax activation function
        self.softmax = nn.LogSoftmax(dim=2)
        self.dropout = nn.Dropout(dropout_p) # Dropout layer for regularization

    def forward(self, currentInput, prevState):
        embdInput = self.embedding(currentInput) # Convert input indices to embeddings
        currEmbd = F.relu(embdInput) # Apply ReLU activation to embeddings
        output, prevState = self.rnn(currEmbd, prevState) # Forward pass through the RNN
        # Apply dropout to the RNN output
        output = self.dropout(output)
        # Apply fully connected layer and softmax activation
        output = self.softmax(self.fc(output)) 
        return output, prevState # Return the output and the updated state

In [17]:
#Data loader function
def dataLoaderFun(dataName,batch_size):
    #Select based on data sets
    if(dataName == 'validation'):
        dataset = MyDataset(data2["source_charToNum"],data2['val_charToNum'])
        return  DataLoader(dataset, batch_size=batch_size, shuffle=True)
    elif(dataName == 'train'):
        dataset = MyDataset(data["source_charToNum"],data['val_charToNum'])
        return DataLoader(dataset, batch_size=batch_size, shuffle=True)
    else:
        dataset = MyDataset(data3["source_charToNum"],data3['val_charToNum'])
        return  DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [18]:
def train(embSize,encoderLayers,decoderLayers,hiddenLayerNuerons,cellType,bidirection,dropout,epochs,batchsize,learningRate,optimizer,tf_ratio):
    
    #Data loader based on training
    dataLoader = dataLoaderFun("train",batchsize)
    
    lossFunction = nn.NLLLoss()
    
    # Initialize encoder and decoder
    encoder = Encoder(data["source_len"],embSize,encoderLayers,hiddenLayerNuerons,cellType,batchsize).to(device)
    decoder = Decoder(data["target_len"],embSize,hiddenLayerNuerons,encoderLayers,cellType,dropout).to(device)
    
    # Initialize optimizer for encoder and decoder
    if(optimizer != 'Adam'):
        encoderOptimizer = optim.NAdam(encoder.parameters(), lr=learningRate)
        decoderOptimizer = optim.NAdam(decoder.parameters(), lr=learningRate)
    else:
        encoderOptimizer = optim.Adam(encoder.parameters(), lr=learningRate)
        decoderOptimizer = optim.Adam(decoder.parameters(), lr=learningRate)
    
    epoch=0
    while(epoch<epochs):
    
        train_accuracy = 0 
        train_loss = 0 

        for batch_num, (sourceBatch, targetBatch) in enumerate(dataLoader):
            
            #hiddenlayers * BatchSize * Neurons
            # Initialize initial state for encoder
            encoderInitialState = encoder.getInitialState()
            
            # Apply bidirectional processing if specified
            if(bidirection == "Yes"):
                reversed_batch = torch.flip(sourceBatch, dims=[1])
            if(bidirection != "No"):
                #Averaging data
                sourceBatch = (sourceBatch + reversed_batch)//2
            
            # Adjust initial state for LSTM cell
            if(cellType == 'LSTM'):
                encoderInitialState = (encoderInitialState, encoder.getInitialState())
                
            # Forward pass through encoder
            encoder_output, encoderCurrentState = encoder(sourceBatch,encoderInitialState)
            
            loss = 0 #Initialize loss
            
            sequenceLen = targetBatch.shape[1]

            Output = []
            
            randNumber = random.random()

            decoderCurrState = encoderCurrentState

            for i in range(0,sequenceLen):
                
                if(i == 0):
                    decoderInput = targetBatch[:, i].reshape(batchsize,1)
                else:
                    if randNumber >= tf_ratio:
                        #Prev decoder output
                        decoderInput = decoderInput.reshape(batchsize, 1)
                    else:
                        #Actual target value
                        decoderInput = targetBatch[:, i].reshape(batchsize, 1)
                
                #Forward pass through decoder
                decoderOutput, decoderCurrState = decoder(decoderInput,decoderCurrState)
                dummy, topIndeces = decoderOutput.topk(1)     
                        
                #Resizing
                decoderOutput = decoderOutput[:, -1, :]
                targetChars = targetBatch[:, i]

                #Find target characters
                targetChars = targetChars.type(dtype=torch.long)

                #Calculating loss               
                loss+=(lossFunction(decoderOutput, targetChars))

                decoderInput = topIndeces.squeeze().detach()
                #Softmax values
                Output.append(decoderInput)
                
            tensor_2d = torch.stack(Output)
            Output = tensor_2d.t()
            
            #Summing up the equal values to find accuracy
            train_accuracy += (Output == targetBatch).all(dim=1).sum().item()
            train_loss += (loss.item()/sequenceLen)
            
            encoderOptimizer.zero_grad()
            decoderOptimizer.zero_grad()
            
            #Backward prop
            loss.backward()
            encoderOptimizer.step()
            decoderOptimizer.step()
            
        epoch=epoch+1
            
        #print("train_accuracy",train_accuracy/512)
        #print("train_loss",train_loss)
        #wandb.log({'train_accuracy':train_accuracy/512})
        #wandb.log({'train_loss':train_loss})
        validationAccuracy(encoder,decoder,batchsize,tf_ratio,cellType,bidirection) #Evaluate valid accuracy

In [ ]:
#For running on test data.
data3 = pre_processing_validation(copy.copy(test_input),copy.copy(test_output))

In [ ]:
def main_fun():
    wandb.init(project ='vanillaRNN')
    params = wandb.config
    with wandb.init(project = 'vanillaRNN', name='embedding'+str(params.embSize)+'cellType'+params.cellType+'batchSize'+str(params.batchsize)) as run:
        train(params.embSize,params.encoderLayers,params.decoderLayers,params.hiddenLayerNuerons,params.cellType,params.bidirection,params.dropout,params.epochs,params.batchsize,params.learningRate,params.optimizer,params.tf_ratio)
    
sweep_params = {
    'method' : 'bayes',
    'name'   : 'VanillaRNN',
    'metric' : {
        'goal' : 'maximize',
        'name' : 'validation_accuracy',
    },
    'parameters' : {
        'cellType' : {'values' : ['GRU','RNN','LSTM'] } ,
        'epochs'  : {'values': [10,15]},
        'batchsize' : {'values' : [32,64]},
        'learningRate' : {'values' : [1e-2,1e-3,1e-4]},
        'embSize':{'values':[16,32,64]},
        'hiddenLayerNuerons'   : {'values' : [64,256,512]},
        'bidirection' : {'values' : ['no','Yes']},
        'dropout' : {'values' : [0,0.2,0.3]},
        'encoderLayers':{'values':[1,5,10]},
        'decoderLayers' : {'values' : [1,5,10]},
        'optimizer':{'values' : ['Adam','Nadam']},
        'tf_ratio' :{'values' : [0.2,0.4,0.5]}
    }
}
sweepId = wandb.sweep(sweep_params,project = 'VanillaRNN')
wandb.agent(sweepId,function =main_fun)